This notebook implements a policy gradient method to tackle the racetrack RL environment. We have a continuous action space so we cannot softmax. We therefore use a neural network to predict mean and standard deviation of a normal distribution. We then sample actions with respect to this distribution to compute loss and train the neural network.

In [1]:
import highway_env
from reinforce import REINFORCE
highway_env.register_highway_envs()

c:\Users\arnoh\anaconda3\envs\pytorch_old_python\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment exit-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\arnoh\anaconda3\envs\pytorch_old_python\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment highway-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\arnoh\anaconda3\envs\pytorch_old_python\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment highway-fast-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\arnoh\anaconda3\envs\pytorch_old_python\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment intersection-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} alre

In [11]:
# Imports
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from copy import deepcopy
import time
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output
import gymnasium as gym
from gymnasium.wrappers import RecordVideo

import matplotlib.pyplot as plt
import pprint

In [3]:
env = gym.make("racetrack-v0", render_mode="rgb_array")

config = {
    "observation": {
        "type": "OccupancyGrid",
        "features": ["presence", "on_road"],
        "grid_size": [[-18, 18], [-18, 18]],
        "grid_step": [5, 5],
        "as_image": False,
        "align_to_vehicle_axes": True
    },
    "action": {
        "type": "ContinuousAction",
        "longitudinal": False,
        "lateral": True,
        "steering_range": [-np.pi / 4, np.pi / 4],  # [rad]
        #"acceleration_range": [-2, 2],  # [m/s²]
        "speed_range": [0, 15],  # [m/s]
    },
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "duration": 300,
    "collision_reward": -1,
    "lane_centering_cost": 4,
    "action_reward": -0.3,
    "controlled_vehicles": 1,
    "other_vehicles": 1,
    "screen_width": 600,
    "screen_height": 600,
    "centering_position": [0.5, 0.5],
    "scaling": 7,
    "show_trajectories": False,
    "render_agent": True,
    "offscreen_rendering": False
}


env.unwrapped.configure(config)
env.reset()

(array([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],
 
        [[1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]], dtype=float32),
 {'speed': 10,
  'crashed': False,
  'action': array([0.21704124], dtype=float32),
  'rewards': {'lane_centering_reward': 1.0,
   'action_reward': 0.21704124,
   'collision_reward': False,
   'on_road_reward': True}})

In [4]:
print(env.spec)
pprint.pprint(env.config)

EnvSpec(id='racetrack-v0', entry_point='highway_env.envs:RacetrackEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=None, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={'render_mode': 'rgb_array'}, namespace=None, name='racetrack', version=0, additional_wrappers=(), vector_entry_point=None)
{'action': {'lateral': True,
            'longitudinal': False,
            'speed_range': [0, 15],
            'steering_range': [-0.7853981633974483, 0.7853981633974483],
            'type': 'ContinuousAction'},
 'action_reward': -0.3,
 'centering_position': [0.5, 0.5],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 300,
 'lane_centering_cost': 4,
 'lane_centering_reward': 1,
 'manual_control': False,
 'observation': {'align_to_vehicle_axes': True,
                 'as_image': False,
                 'features': ['presence', 'on_road'],
                 'grid_size': [[-18, 18], [-18, 18]],
              

c:\Users\arnoh\anaconda3\envs\pytorch_old_python\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.config to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.config` for environment variables or `env.get_wrapper_attr('config')` that will search the reminding wrappers.
  logger.warn(


In [5]:
#The baseline agent to compare performances

class RandomAgent:
    def __init__(self, observation_space, action_space):
        self.action_space = action_space
        return

    def get_action(self, state, **kwargs):
        return self.action_space.sample()

    def update(self, *data):
        pass

In [6]:
def eval_agent(agent, env, n_sim=10):
    """
    Monte Carlo evaluation of the agent.

    Repeat n_sim times:
        * Run the agent policy until the environment reaches a terminal state (= one episode)
        * Compute the sum of rewards in this episode
        * Store the sum of rewards in the episode_rewards array.
    """
    env_copy = deepcopy(env)
    episode_rewards = np.zeros(n_sim)
    for i in range(n_sim):
        state, _ = env_copy.reset()
        reward_sum = 0
        done = False
        while not done:
            action = agent.get_action(state, epsilon=0)
            #print(action)
            state, reward, terminated, truncated, _ = env_copy.step(action)
            reward_sum += reward
            done = terminated or truncated
        episode_rewards[i] = reward_sum
    return episode_rewards

In [7]:

#agent = RandomAgent(env.observation_space, env.action_space)

def run_one_episode(env, agent, display=True):
    display_env = deepcopy(env)
    done = False
    state, _ = display_env.reset()

    rewards = 0

    while not done:

        action = agent.get_action(state, epsilon=0)
        state, reward, done, _, _ = display_env.step(action)
        rewards += reward
        if display:
            clear_output(wait=True)
            plt.imshow(display_env.render())
            plt.show()
    if display:
        display_env.close()
    print(f'Episode length {rewards}')

# run_one_episode(env, agent, display=False)
# print(f'Average over 5 runs : {np.mean(eval_agent(agent, env))}')

In [8]:
def train(env, agent, N_episodes, eval_every=100, reward_threshold=400, n_eval=10):
    total_time = 0
    for ep in range(N_episodes):
        done = False
        state, _ = env.reset()
        while not done:
            action = agent.get_action(state, epsilon = 0)
            #print(action)
            next_state, reward, terminated, truncated, _ = env.step(action)
            agent.update(state, action, reward, terminated, next_state)

            state = next_state

            done = terminated or truncated
            total_time += 1

        if ((ep+1)% eval_every == 0):
            mean_reward = np.mean(eval_agent(agent, env, n_sim=n_eval))
            print("episode =", ep+1, ", reward = ", mean_reward)
            if mean_reward >= reward_threshold:
                break

    return

In [9]:


# action_space = env.action_space
# observation_space = env.observation_space

# gamma = .95
# episode_batch_size = 30
# learning_rate = 1e-2

# agent = REINFORCE(
#         action_space,
#         observation_space,
#         gamma,
#         episode_batch_size,
#         learning_rate,
#         )
# N_episodes = 3000

gamma = 0.8
batch_size = 32
buffer_capacity = 15_000
#update_target_every = 32

epsilon_start = 0.4 #0.8
decrease_epsilon_factor = 50 #100
epsilon_min = 0.02

# hidden_size = 256
input_channels = 2

learning_rate = 5e-4

agent = REINFORCE(env,
            gamma,
            batch_size,
            buffer_capacity,
            epsilon_start,
            decrease_epsilon_factor,
            epsilon_min,
            learning_rate,
            input_channels
        )
N_episodes = 100


#print("mean reward before training = ", np.mean(eval_agent(agent, env, 1)))
# Run the training loop
train(env, agent, N_episodes, eval_every=50,)

# Evaluate the final policy
print("mean reward after training = ", np.mean(eval_agent(agent, env, 20)))
print(eval_agent(agent, env, 20))

mean reward after training =  11.03581000881692
[10.15108582 10.0008364  26.21282753 22.95331702 29.98349032 13.66242371
 31.7738181   7.0698197  15.46031989  1.04911098]


In [12]:
# env = RecordVideo(
#     env, video_folder="racetrack_PG/videos", episode_trigger=lambda e: True
# )
# env.unwrapped.set_record_video_wrapper(env)

# for video in range(10):
#     done = truncated = False
#     obs, info = env.reset()
#     while not (done or truncated):
#         # Predict
#         action, _states = model.predict(obs, deterministic=True)
#         # Get reward
#         obs, reward, done, truncated, info = env.step(action)
#         # Render
#         env.render()
# env.close()

DependencyNotInstalled: moviepy is not installed, run `pip install moviepy`